
# I. Présentation de l'API LégiFrance :

Au total : ~70 requêtes API classées en 6 grandes familles.





### Familles de requêtes développées



#### 1. **Consult Controller**
- **Objectif** : Récupérer des informations spécifiques législatives ou réglementaires (textes, articles, etc.).
- **Fonctionnalités** :
  - Obtenir un article spécifique via son `id`.
  - Récupérer des détails sur une loi ou un texte réglementaire précis.



#### 2. **Chrono Controller**
- **Objectif** : Accéder à l'historique et à l'évolution chronologique des textes.
- **Fonctionnalité** :
  - Suivre les modifications ou révisions d’un texte au fil du temps.



#### 3. **List Controller**
- **Objectif** : Fournir des listes de documents basées sur des critères spécifiques.
- **Fonctionnalités** :
  - Lister tous les articles d'un code particulier.
  - Récupérer la liste des décrets associés à une loi.





### Familles de requêtes non développées



#### 4. **Suggest Controller**
- **Objectif** : Fournir des suggestions basées sur une recherche partielle.
- **Fonctionnalités** :
  - Rechercher un texte via des mots-clés ou des fragments.



#### 5. **Misc Controller**
- **Objectif** : Récupérer des métadonnées ou des informations auxiliaires.
- **Fonctionnalités** :
  - Obtenir des données complémentaires sur les textes ou documents.



#### 6. **Search Controller**
- **Objectif** : Effectuer des recherches globales sur les textes, articles, codes, etc., en fonction de mots-clés ou d'autres critères.
- **Fonctionnalités** :
  - Rechercher des textes ou articles pertinents par mots-clés.
  - Filtrer les résultats en fonction de critères spécifiques.


### Listes to do:
- subdiviser balise html
- passer apres en .py

Fait:
- Développer en fonction
- Quand long télécharger json
- pas plus que deux boucles for (pas de moyen trouvé)
- plus utiliser pandas
- dataframe (flatten)
- test sur cmf
- ajout contenu article N vs N-1

# 1 - LégiFrance API connect

## 1.0 - Credential input

In [1]:
client_id = '2bd5c3ac-5128-4527-b74f-5d3e90e47efb'
client_secret = 'de5cf33d-fbee-4368-8fe9-e803490fe096'

In [3]:
import getpass

## 1.1 - API Get access token

In [2]:
import sys
print(sys.version)

3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:41:52) [Clang 15.0.7 ]


In [48]:
import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session
import json

In [49]:
def get_token():
    token_url = 'https://sandbox-oauth.piste.gouv.fr/api/oauth/token'
    #inject cred 
    token_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'openid'}
    response = requests.post(token_url, data=token_data)
    response.raise_for_status()  # vérif  erreurs
    # récup  jeton
    token_info = response.json()
    global access_token
    access_token = token_info['access_token']
    return access_token


In [50]:
get_token()

'kWOPsudicJVSWuZZ3kk2csoXU2Z6H7fMi0sKh2sEI7L4BF4TFKmPhP'

## 1.2 - Ping/Pong request Test

In [43]:
import requests

def ping_pong_test():
    headers_1 = {'accept': 'text/plain', 'Authorization': 'Bearer ' + access_token}
    output = requests.get("https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/ping", headers=headers_1).text
    display(output)

In [51]:
ping_pong_test()

'pong'

# 2 - Requêtes - API calls



### 2.1 - Contenu Article - avec Eli (Fait)






Eli - Identifiant Européen de la Législation:

- Permet d'identifier de façon unique les documents législatifs et réglementaires au sein de la base de données de Légifrance.

- Déclinaison :

  - /eli : Préfixe indiquant qu'il s'agit d'un identifiant ELI.
  - /decret : Type de texte (ici un décret) peut aussi être un arrêté, une loi, ou d'autres types d'actes normatifs...
  - /2021/7/13 : Date de publication ou adoption (13 juillet 2021 dans cet exemple).
  - /PRMD2117108D : Numéro unique du décret
  - /jo : Indication que le document a été publié dans le Journal Officiel.
  - /article_1 : Référence à un article spécifique dans le document (ici l'article 1).


In [8]:
def get_article_byELI(idEliOrAlias):
  headers_2 = {"accept": "application/json","Content-Type": "application/json", 'Authorization': 'Bearer ' + access_token}
  data = {"idEliOrAlias": idEliOrAlias } # exemple: "/eli/decret/2021/7/13/PRMD2117108D/jo/article_1"
  url= "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticleWithIdEliOrAlias"
  response = requests.post(url, headers=headers_2,json=data)
  response_json = response.json()
  return {
        "etat": response_json['article']['textTitles'][1]['etat'],
        "id": response_json['article']['id'],
        "titresTM": response_json['article']['context']['titresTM'],
        "titreLong": response_json['article']['textTitles'][1]["titreLong"],
        "texte": response_json['article']['texte']
    }


In [9]:
get_article_byELI("/eli/ordonnance/2020/12/9/ECOT2024817R/jo/article_1") 
# exemple: /eli/decret/2021/7/13/PRMD2117108D/jo/article_1

{'etat': 'VIGUEUR',
 'id': 'JORFARTI000042636235',
 'titresTM': [],
 'titreLong': 'Ordonnance n° 2020-1544 du 9 décembre 2020 renforçant le cadre de la lutte contre le blanchiment de capitaux et le financement du terrorisme applicable aux actifs numériques',
 'texte': "L'article L. 54-10-3 du code monétaire et financier est ainsi modifié : 1° Le premier alinéa est remplacé par les dispositions suivantes : « Avant d'exercer leur activité, les prestataires des services mentionnés aux 1° à 4° de l'article L. 54-10-2 établis en France ou fournissant ces services en France, sont enregistrés par l'Autorité des marchés financiers, qui vérifie si : » ; 2° Le 3° est remplacé par les dispositions suivantes : « 3° Les prestataires sont établis en France ou dans un autre Etat membre de l'Union européenne ou partie à l'accord sur l'Espace économique européen ; » 3° Après le 3°, il est inséré un 4° ainsi rédigé : « 4° Pour les services mentionnés aux 1° et 2° de l'article L. 54-10-2, elle vérifie ég

### 2.2 - Contenu texte type CODE  /consult/code (Fait - pas utilisé)


Fonction : Permet d'obtenir le contenu complet d'un code juridique


In [10]:
import json
from IPython.display import HTML
import urllib.parse

def download_json(variable, filename="data.json"):
    json_str = json.dumps(variable)
    json_encoded = urllib.parse.quote(json_str)
    href = f'<a href="data:text/json;charset=utf-8,{json_encoded}" download="{filename}">Télécharger {filename}</a>'
    return HTML(href)


In [11]:
def get_code(textId,sctCid,abrogated, date):
  headers = {"Authorization": "Bearer " + access_token , "Content-Type": "application/json" }
  BASE_URL = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/code"
  data = {
    "textId": textId,         #"LEGITEXT000006073984",  # Code des assurances
    "sctCid": sctCid,         #"LEGIARTI000048769089",  # ID article
    "abrogated": abrogated,   # False,                # exclure les textes abrogés
    "date":date  #"2024-11-15",              # Date de référence
    }
  response = requests.post(BASE_URL, json=data, headers=headers)
  response_json = response.json()
  return response_json

In [ ]:
get_code("LEGITEXT000006073984", "LEGIARTI000042644804","False", "2023-04-08" )

### 2.3 - Contenu  d'un article en fonction des versions /consult/getArticleByCid (Fait)


Objectif : Alimenter le tableau avec le contenu des articles a travers deux fonctions:
- La première récupère le contenu de l'article a la date d'entrée en vigueur
- La deuxieme récupère le contenu de l'article a la date de fin



#### 2.1.3.0 - Test function & debug

Fonction qui test la récupération de la liste des articles par leur identifiant Cid en retournant le contenu et le message status_code

Exemples identifiant Cid:

- LEGIARTI000028443554 https://www.legifrance.gouv.fr/codes/article_lc/LEGIARTI000028443554/2014-01-01

- LEGIARTI000028443552 https://www.legifrance.gouv.fr/codes/article_lc/LEGIARTI000028443552/2014-01-01



In [12]:
def getArticle_ByCid_allversions_test(article_cid):
  API_URL = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticleByCid'
  headers = {'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json'}
  data = { 'cid': article_cid }
  response = requests.post(API_URL, json=data, headers=headers)
  response_json = response.json()
  return {"contenu": response.json(),
          "reponse status code ":response.status_code }

In [13]:
getArticle_ByCid_allversions_test(article_cid= "LEGIARTI000038101111")

{'contenu': {'executionTime': 0, 'listArticle': []},
 'reponse status code ': 200}

#### 2.1.3.1 - Get the last version of an article (by Cid)

In [14]:
from datetime import datetime

def format_date(timestamp_ms):
    timestamp_s = timestamp_ms // 1000  # Convertir millisecondes en secondes
    return datetime.utcfromtimestamp(timestamp_s).strftime('%Y-%m-%d')

def getArticle_ByCid_last_version(article_cid):
  API_URL = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticleByCid'
  headers = {'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json'}
  data = { 'cid': article_cid }
  response = requests.post(API_URL, json=data, headers=headers)
  response_json = response.json()
  return {"Date Debut:" : format_date(response.json()['listArticle'][0]['dateDebut']),
          "Date Fin:": format_date(response.json()['listArticle'][0]['dateFin']),
          "Version:": response.json()['listArticle'][0]['versionArticle'],
          "Texte:": response.json()['listArticle'][0]['texte']
          }

In [16]:
getArticle_ByCid_last_version("LEGIARTI000047390103")

{'Date Debut:': '2023-04-03',
 'Date Fin:': '2999-01-01',
 'Version:': '1.0',
 'Texte:': "La demande d'agrément est accompagnée des éléments suivants : 1° La convention constitutive signée par les représentants légaux de l'ensemble des entreprises d'assurance qui commercialisent, au moment du dépôt de la demande, des produits d'assurance contre les risques climatiques en agriculture bénéficiant de l'aide prévue au deuxième alinéa de l'article L. 361-4 du code rural et de la pêche maritime ; 2° Une analyse économique de l'impact du groupement sur le marché de la couverture des risques climatiques au regard de l'intensité concurrentielle du secteur assurantiel concerné et des gains économiques attendus pour les exploitants agricoles ; 3° L'avis de l'Autorité de la concurrence mentionné au III de l'article L. 442-1-2 ; 4° Un compte-rendu exhaustif ainsi que l'ensemble des contributions écrites de la consultation publique mentionnée à l'article L. 442-1-2."}

#### 2.1.3.2 - Get all versions of an artcile (by Cid )

In [17]:
import requests

def get_articles_by_cid_all_versions(article_cid):
    API_URL = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticleByCid'
    headers = {'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json'}
    data = {'cid': article_cid}

    response = requests.post(API_URL, json=data, headers=headers)
    response_json = response.json()

    # Liste des articles récupérés
    articles = response_json.get('listArticle', [])

    # Résultats formatés
    formatted_articles = []
    for article in articles:
        formatted_articles.append({
            "Date Debut": format_date(article.get('dateDebut')),
            "Date Fin": format_date(article.get('dateFin')),
            "Version": article.get('versionArticle'),
            "Texte": article.get('texte')
        })

    return formatted_articles


In [20]:
get_articles_by_cid_all_versions("LEGIARTI000028443554")

[{'Date Debut': '2023-12-31',
  'Date Fin': '2999-01-01',
  'Version': '4.0',
  'Texte': "Le taux des contributions mentionnées à l'article L. 421-4-1 est fixé par arrêté du ministre chargé des assurances dans les limites suivantes : 1° Pour la contribution des assurés, ce taux est compris entre 0 % et 2 % des primes mentionnées au 1° du même article ; 2° Pour la contribution des entreprises d'assurance, ce taux est compris entre 0 % et 1 % des primes ou cotisations mentionnées au 2° du même article L. 421-4-1 ; 3° (abrogé) ; 4° Pour la contribution des responsables d'accidents non assurés, ce taux est fixé à 10 % des indemnités restant à leur charge. Toutefois, ce taux peut être ramené à 5 % lorsque l'accident a été provoqué par un véhicule utilisé par l'Etat ou par un Etat étranger. Il est également ramené à 5 % des indemnités restant à leur propre charge pour les bénéficiaires d'une assurance avec franchise."},
 {'Date Debut': '2018-12-31',
  'Date Fin': '2023-12-31',
  'Version': '

#### 2.1.3.3 - Get new article content by cid and implementation date for table

Fonction pour remplissage de la premiere colonne du tableau qui correspond a la nouvelle version de l'article.

La fonction prend en entrée deux paramètres l'identifiant de l'article (Cid) et la date d'entrée en vigueur (start_date).

Attention : La fonction ne fonctionne pas pour tous les articles.  

In [22]:
import requests
from datetime import datetime

def get_article_text_by_cid_and_implementDate(article_cid, start_date):
    API_URL = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticleByCid'
    headers = {'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json'}
    data = {'cid': article_cid}

    response = requests.post(API_URL, json=data, headers=headers)
    response_json = response.json()

    # Convertir la date en millisecondes depuis l'époque Unix
    start_date_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp() * 1000)

    # Parcourir les articles du tableau pour trouver une correspondance
    articles = response_json.get('listArticle', []) # listArticle output json key de la requete
    for article in articles:
        if article.get('dateDebut') == start_date_timestamp:
            return article.get('texte', "Texte non disponible")

    # Si aucun article ne correspond
    return "Aucun article trouvé pour la date de début donnée."


In [24]:
get_article_text_by_cid_and_implementDate(article_cid = "LEGIARTI000038101111",  start_date= "2018-07-01")
# Ok : "LEGIARTI000028443554",  "2018-07-01"
# KO : "LEGIARTI000048656110"  "2023-12-23"

'Aucun article trouvé pour la date de début donnée.'

#### 2.1.3.3 - Get old article content by cid and end date for table

In [21]:
import requests
from datetime import datetime

def get_article_text_by_cid_and_end_date(article_cid, end_date):
    API_URL = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticleByCid'
    headers = {'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json'}
    data = {'cid': article_cid}

    response = requests.post(API_URL, json=data, headers=headers)
    response_json = response.json()

    # Convertir la date en millisecondes depuis l'époque Unix
    end_date_timestamp = int(datetime.strptime(end_date, "%d-%m-%Y").timestamp() * 1000) #(ko: %Y-%m-%d ) (ko:"%d-%m-%Y" )

    # Parcourir les articles pour trouver une correspondance
    articles = response_json.get('listArticle', [])
    for article in articles:
        if article.get('dateFin') == end_date_timestamp:
            return article.get('texte', "Texte non disponible")

    # Si aucun article ne correspond
    return "Aucun article trouvé pour la date de début donnée."

In [22]:
get_article_text_by_cid_and_end_date("LEGIARTI000048243369","25-10-2023") #KO :"LEGIARTI000046918899","2023-11-01"

'Aucun article trouvé pour la date de début donnée.'

### 2.4 - Contenu d'un dossier législatif JORF /consult/dossierLegislatif (Fait)



Récupère le contenu d'un dossier legislatif par son identifiant.

Input : Identifiant technique du dossier législatif "id": "JORFDOLE000038049286"



In [25]:
import requests
import json

def dossierLegislatif(id): #JORFDOLE000038049286 
	headers = {'accept': 'application/json','Content-Type': 'application/json','Authorization': 'Bearer ' + access_token}
	url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/dossierLegislatif'
	data = {"id":id}
	reponse = requests.post(headers = headers , url= url, json= data)
	reponse_json = reponse.json()
	return reponse_json

In [52]:
dossierLegislatif("JORFTEXT000030677073")

{'executionTime': 0, 'dereferenced': False, 'dossierLegislatif': None}

### 2.5 - Contenu texte fonds LEGI /consult/legiPart ( Fait )



Récupère des informations précises sur des articles de loi, des codes ou d'autres textes juridiques a partir d'identifiant ("textId") et de sa date de vigueur:

In [26]:
def legipart(textId, date): # LEGITEXT000006075116 2021-04-15
  url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/legiPart'
  headers = {'accept': 'application/json','Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
  data = { #"searchedString":"constitution 1958",
        "date": date,
        "textId":textId }
  response = requests.post(url= url, headers =headers, json= data )
  return response.json()

In [ ]:
legipart("LEGIARTI000038101111", "2021-04-15") 

### 2.6 - Consult JORF

Pour trouver et récup JORF ordonnance
Exemple: https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000042636234


In [54]:
def getJOFR(textCid :str): #JORFTEXT000042636234
  url= 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/jorf'
  headers= {'accept': 'application/json',
		'Content-Type': 'application/json',
		'Authorization': 'Bearer ' + access_token}
  data= {"searchedString":"","textCid":textCid}
  response = requests.post( headers =headers, url= url, json= data )
  response_json = response.json()
  return response_json

In [ ]:
getJOFR("JORFTEXT000030677073")

### 2.7 - Contenu d'un article /consult/getArticle (Fait)


Récupère un article par son identifiant (article id)

id : ID unique de la version de l'article pour accéder directement à une version particulière (historique ou active).



#### Get article and metadata


In [29]:
def getArticle_metadata(id : str):
  headers= {'accept': 'application/json','Content-Type': 'application/json','Authorization': 'Bearer ' + access_token}
  url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle'
  data = {"id":id}
  response= requests.post(url=url,  headers=headers, json = data)
  return {"num":  response.json()["article"]['num'],
          "etat": response.json()["article"]['etat'],
         "Text" : response.json()["article"]['texte']}


In [30]:
getArticle_metadata("LEGIARTI000048844359")
# OK: LEGIARTI000048243369, LEGIARTI000048656110 , LEGIARTI000006812142

{'num': 'L421-4-1',
 'etat': 'VIGUEUR',
 'Text': "Les contributions pour l'alimentation du fonds de garantie mentionnées à l'article L. 421-4 sont ainsi définies : 1° La contribution des assurés est assise sur toutes les primes ou cotisations nettes qu'ils versent aux entreprises d'assurance pour l'assurance des risques de responsabilité civile résultant d'accidents causés par les véhicules terrestres à moteur et des remorques ou semi-remorques des véhicules lorsque le risque est situé sur le territoire de la République française. Elle est perçue par les entreprises d'assurance suivant les mêmes règles et sous les mêmes garanties et sanctions que la taxe sur les conventions d'assurance prévue à l'article 991 du code général des impôts. Elle est recouvrée mensuellement par le fonds de garantie ; 2° La contribution des entreprises d'assurance est assise sur toutes les primes ou cotisations nettes qu'elles perçoivent pour l'assurance des risques de responsabilité civile résultant d'accide

#### Get content of previous version of an article (function for the 2de column)


In [31]:
def getArticle_prev_vers(id : str):
  headers= {'accept': 'application/json','Content-Type': 'application/json','Authorization': 'Bearer ' + access_token}
  url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle'
  data = {"id":id}
  response= requests.post(url=url,  headers=headers, json = data)
  nouvel_id= response.json()['article']['articleVersions'][-2]['id']
  nouveau_data = {"id":nouvel_id}
  response_nouvelle= requests.post(url=url,  headers=headers, json = nouveau_data)
  response_json= response_nouvelle.json()
  return response_json["article"]['texte']

In [32]:
import requests
def getArticle_prev_vers(id: str):
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + access_token
    }
    url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle'

    try:
        # Requête initiale pour obtenir les versions de l'article
        data = {"id": id}
        response = requests.post(url=url, headers=headers, json=data)

        # Vérification du code de statut HTTP
        if response.status_code != 200:
            return "KO status_code"

        # Extraction des données de l'article
        article = response.json().get("article", {})
        article_versions = article.get("articleVersions", [])

        # Vérification de l'existence d'une version précédente
        if len(article_versions) < 2:
            return "KO version précédente"

        # Récupération de l'ID de la version précédente
        nouvel_id = article_versions[-2].get("id")
        if not nouvel_id:
            return "KO absence ID de la version précédente "

        # Requête pour la version précédente
        nouveau_data = {"id": nouvel_id}
        response_nouvelle = requests.post(url=url, headers=headers, json=nouveau_data)

        # Vérification du code de statut HTTP pour la seconde requête
        if response_nouvelle.status_code != 200:
            return "KO status_code HTTP seconde requête"

        # Extraction des données de la version précédente
        response_json = response_nouvelle.json()
        texte = response_json.get("article", {}).get("texte")

        # Vérification si le texte de l'article existe
        return texte if texte else "KO"

    except Exception as e:
        # Gestion des exceptions (problème de connexion, JSON invalide, etc.)
        print(f"Erreur : {e}")
        return "KO"


In [72]:
getArticle_prev_vers("LEGIARTI000048844359")
# OK: LEGIARTI000048243369, LEGIARTI000048656110 , LEGIARTI000006812142

"Les contributions pour l'alimentation du fonds de garantie mentionnées à l'article L. 421-4 sont ainsi définies : 1° La contribution des assurés est assise sur toutes les primes ou cotisations nettes qu'ils versent aux entreprises d'assurance pour l'assurance des risques de responsabilité civile résultant d'accidents causés par les véhicules terrestres à moteur et des remorques ou semi-remorques des véhicules lorsque le risque est situé sur le territoire de la République française. Elle est perçue par les entreprises d'assurance suivant les mêmes règles et sous les mêmes garanties et sanctions que la taxe sur les conventions d'assurance prévue à l'article 991 du code général des impôts. Elle est recouvrée mensuellement par le fonds de garantie ; 2° La contribution des entreprises d'assurance est proportionnelle aux primes ou cotisations du dernier exercice, accessoires et rappels compris et annulations déduites, relatives à l'assurance des véhicules terrestres à moteur et des remorque

#### Get content new version of an article (function for the 1srt column)


In [34]:
def getArticle(id : str):
  headers= {'accept': 'application/json','Content-Type': 'application/json','Authorization': 'Bearer ' + access_token}
  url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle'
  data = {"id":id}
  response= requests.post(url=url,  headers=headers, json = data)
  return response.json()["article"]['texte']


In [35]:
getArticle("LEGIARTI000048844359")

"Les contributions pour l'alimentation du fonds de garantie mentionnées à l'article L. 421-4 sont ainsi définies : 1° La contribution des assurés est assise sur toutes les primes ou cotisations nettes qu'ils versent aux entreprises d'assurance pour l'assurance des risques de responsabilité civile résultant d'accidents causés par les véhicules terrestres à moteur et des remorques ou semi-remorques des véhicules lorsque le risque est situé sur le territoire de la République française. Elle est perçue par les entreprises d'assurance suivant les mêmes règles et sous les mêmes garanties et sanctions que la taxe sur les conventions d'assurance prévue à l'article 991 du code général des impôts. Elle est recouvrée mensuellement par le fonds de garantie ; 2° La contribution des entreprises d'assurance est assise sur toutes les primes ou cotisations nettes qu'elles perçoivent pour l'assurance des risques de responsabilité civile résultant d'accidents causés par les véhicules terrestres à moteur 

In [36]:
import requests

def getArticle(id: str):
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + access_token
    }
    url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle'
    data = {"id": id}

    try:
        response = requests.post(url=url, headers=headers, json=data)
        # Vérification du statut HTTP
        if response.status_code != 200:
            return "KO Status Code"

        # Extraction des données
        article = response.json().get("article", {})

        # Vérification si le texte de l'article est présent
        if "texte" in article:
            return article["texte"]
        else:
            return "KO article pas trouvé"

    except Exception as e:
        # Gestion des exceptions (erreur de connexion, JSON invalide, etc.)
        print(f"Erreur : {e}")
        return "KO autre"


### 2.8 - Extrait d'une version d'un texte  /chrono/textCidAndElementCid (Fait - pas utilisé)

Récupère un extrait (section ou article) d'une version spécifique d'un texte à partir des identifiants du texte (textCid) et de l'extrait section ou article: "elementCid"

Remarque : absence de contenu d'article dans la réponse

In [37]:
def textCidAndElementCid(textCid, elementCid): # text cid: LEGITEXT000006072050 Element Cid LEGIARTI000006901817
  url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCidAndElementCid"
  headers = {
      'accept': 'application/json',
      'Content-Type': 'application/json',
      'Authorization': 'Bearer ' + access_token}
  data = { "textCid": textCid, # code du travail
    "elementCid": elementCid # Article L2262-8
  }
  response = requests.post(url, headers=headers, json= data)
  response_json= response.json()
  return response_json

In [38]:
# Test 1: "textCid": "LEGITEXT000006070721" Code Civil, "elementCid": "LEGIARTI000038033084" -> NA
# Test 2: "textCid": "LEGITEXT000006070721" Code Civil , "elementCid": "LEGIARTI000006417324" -> NA
# Test 3 : "textCid": "LEGITEXT000006072050" Code du Travail + Article L1221-1 "elementCid": "LEGIARTI000006901817" -> ok
# https://www.legifrance.gouv.fr/codes/article_lc/LEGIARTI000006900839

In [39]:
textCidAndElementCid("LEGITEXT000006072050", 'LEGIARTI000006901817')

{'executionTime': 0,
 'regroupements': [{'title': '2008',
   'versions': {'2008-05-01': {'dateDebut': 1209600000000,
     'isEndVersion': False,
     'articlesModificateurs': {'JORFTEXT000000465978': {'title': 'Ordonnance 2007-329 2007-03-12 JORF 13 mars 2007',
       'id': 'JORFTEXT000000465978',
       'nature': 'CODE',
       'dateDebutCible': 32472144000000,
       'actions': {'CODIFICATION': {'action': 'CODIFICATION',
         'parents': {'LEGISCTA000006189538': {'name': 'Section 2 : Information et communication.',
           'id': 'LEGISCTA000006189538',
           'cid': 'LEGISCTA000006189538',
           'cidText': 'LEGITEXT000006072050',
           'nature': 'CODE',
           'dateDebut': 1209600000000,
           'articlesCibles': {'LEGIARTI000006901817': {'name': 'L2262-8',
             'id': 'LEGIARTI000006901817',
             'cid': 'LEGIARTI000006901817',
             'idParent': 'LEGISCTA000006189538',
             'cidParent': 'LEGISCTA000006189538',
             'cid

### 2.9 - Récupère une version (plage de dates) d'un texte (textCid) et version en vigueur (date) (Fait)




Exemple:

- Consulter le texte spécifié par textCid ("LEGITEXT000006070721").

- Voir les versions de ce texte entre 2015 et 2023


#### 2.9.1 - Test function output status code and json file keys

In [40]:
def get_text_modif_byDateslot_textCid_test(textCid, startYear, endYear): #LEGITEXT000006073984 code des assurances
  headers = { 'accept': 'application/json', 'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
  data = {  "endYear": endYear,  #"dateConsult": "2021-04-15",
  "startYear": startYear,
  "textCid": textCid  }
  url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCid'
  response = requests.post(url, headers=headers, json=data)
  response_json= response.json()
  return {"status code : ": print(response),
  "keys : ": response_json.keys()}

In [ ]:
get_text_modif_byDateslot_textCid_test(textCid= "LEGITEXT000006072026",
                                  startYear= "2019",
                                  endYear= "2021" )

<Response [200]>


{'status code : ': None,
 'keys : ': dict_keys(['executionTime', 'regroupements', 'datePublication'])}

#### 2.9.2 - Central Function to extract code content

In [81]:
def get_text_modif_byDateslot_textCid_extract_content(textCid, startYear, endYear): #LEGITEXT000006073984 code des assurances
  headers = { 'accept': 'application/json', 'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
  data = {  "endYear": endYear,  #"dateConsult": "2021-04-15",
  "startYear": startYear,
  "textCid": textCid  }
  url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCid'
  response = requests.post(url, headers=headers, json=data)
  response_json= response.json()
  return  response_json

In [100]:
data_json_2_2_3 = get_text_modif_byDateslot_textCid_extract_content(textCid= "LEGITEXT000006072026",
                                  startYear= "2019",
                                  endYear= "2021")

In [ ]:
data_json_2_2_3

#### 2.9.3 - Data pipeline 1: Data Print

In [45]:
from datetime import datetime as dt
# Data cleaning
annee_recherchee = "2023"
resultat = ""

# Vérifier la présence du champ 'regroupements' et l’analyser
for group_1 in data_json_2_2_3['regroupements']:  # Parcours regroupement d'années : list de dictionnaires
    if group_1['title'] == annee_recherchee:  # Filtre pour l'année recherchée
        print(f"Année : {group_1['title']}")
        #resultat = resultat+ group_1['title'] + "n"

        # Parcours de chaque version de texte pour l'année courante
        for version_date, version_data in group_1['versions'].items():
            # Convertir le timestamp en une date lisible
            date_debut = dt.fromtimestamp(version_data['dateDebut'] / 1000).strftime('%Y-%m-%d')

            print(f"  Version du {version_date}:")
            print(f"    Date de début cible entrée en vigueur: {date_debut}")
            print(f"    Est la dernière version : {'Oui' if version_data['isEndVersion'] else 'Non'}")

            # Affichage des articles modificateurs
            print("    Articles modificateurs :")
            for article_id, article_data in version_data['articlesModificateurs'].items():
                print(f"      - ID : {article_id}")
                print(f"        Titre : {article_data['title']}")
                print(f"        Nature : {article_data['nature']}")
                date_debut_cible = dt.fromtimestamp(article_data['dateDebutCible'] / 1000).strftime('%Y-%m-%d')
                print(f"        Date de début cible : {date_debut_cible}")

                # Affichage des actions et des parents si disponibles
                if 'actions' in article_data:
                    for action_type, action_details in article_data['actions'].items():
                        print(f"        Action : {action_type}")

                        if 'parents' in action_details:
                            print("        Parents :")
                            for parent_id, parent_data in action_details['parents'].items():
                                print(f"          - ID parent  : {parent_id}")
                                print(f"            Nom parent : {parent_data['name']}")
                                print(f"            CID parent : {parent_data['cid']}")
                                if 'articlesCibles' in parent_data:
                                    print("            Articles cibles :")
                                    for cible_id, cible_data in parent_data['articlesCibles'].items():
                                        print(f"              - ID :{cible_id} - Article N° {cible_data['name']}")
                                        print(f"                    Date de début : {dt.fromtimestamp(cible_data['dateDebut'] / 1000).strftime('%Y-%m-%d')}")
                                        print(f"                    Date de fin : {dt.fromtimestamp(cible_data['dateFin'] / 1000).strftime('%Y-%m-%d')}")



Année : 2023
  Version du 2023-12-31:
    Date de début cible entrée en vigueur: 2023-12-31
    Est la dernière version : Non
    Articles modificateurs :
  Version du 2023-12-30:
    Date de début cible entrée en vigueur: 2023-12-30
    Est la dernière version : Non
    Articles modificateurs :
  Version du 2023-12-08:
    Date de début cible entrée en vigueur: 2023-12-08
    Est la dernière version : Non
    Articles modificateurs :
  Version du 2023-12-01:
    Date de début cible entrée en vigueur: 2023-12-01
    Est la dernière version : Non
    Articles modificateurs :
  Version du 2023-10-26:
    Date de début cible entrée en vigueur: 2023-10-26
    Est la dernière version : Non
    Articles modificateurs :
  Version du 2023-10-25:
    Date de début cible entrée en vigueur: 2023-10-25
    Est la dernière version : Non
    Articles modificateurs :
  Version du 2023-10-15:
    Date de début cible entrée en vigueur: 2023-10-15
    Est la dernière version : Non
    Articles modificat

#### 2.9.4 - Data pipeline 2: To pd.DataFrame

In [46]:
from datetime import datetime

def convert_timestamp(timestamp):
    """
    Convertit un timestamp en millisecondes en une date lisible (YYYY-MM-DD).
    """
    if timestamp is None:
        return None
    return datetime.utcfromtimestamp(timestamp / 1000).strftime('%Y-%m-%d')


In [47]:
import pandas as pd
def transform_json_to_dataframe(data):
    """
    Transforme le JSON en DataFrame structuré avec les colonnes demandées.
    """
    rows = []

    for regroupement in data["regroupements"]:
        year = regroupement["title"]
        for version, version_data in regroupement["versions"].items():
            date_debut_version = convert_timestamp(version_data.get("dateDebut"))
            is_last_version = version_data.get("isEndVersion", False)
            articles_modificateurs = version_data.get("articlesModificateurs", {})

            for article_id, article_data in articles_modificateurs.items():
                title_article = article_data.get("title")
                nature_article = article_data.get("nature")
                date_debut_cible_article = convert_timestamp(article_data.get("dateDebutCible"))
                actions = article_data.get("actions", {})

                for action_type, action_data in actions.items():
                    parents = action_data.get("parents", {})
                    for parent_id, parent_data in parents.items():
                        parent_name = parent_data.get("name")
                        parent_cid = parent_data.get("cid")
                        articles_cibles = parent_data.get("articlesCibles", {})

                        for article_cible_id, article_cible_data in articles_cibles.items():
                            article_cible_name = article_cible_data.get("name")
                            article_cible_date_debut = convert_timestamp(article_cible_data.get("dateDebut"))
                            article_cible_date_fin = convert_timestamp(article_cible_data.get("dateFin"))

                            # Ajouter une ligne au tableau
                            rows.append({
                                "Année": year,
                                "Version du": version,
                                "Date de début cible d'entrée en vigueur": date_debut_version,
                                "Est la dernière version": is_last_version,
                                "ID Article Modificateur": article_id,
                                "Titre Article Modificateur": title_article,
                                "Nature Article Modificateur": nature_article,
                                "Date de début cible Article Modificateur": date_debut_cible_article,
                                "Action Article Modificateur": action_type,
                                "ID Parent": parent_id,
                                "Nom Parent": parent_name,
                                "CID Parent": parent_cid,
                                "ID Article Cible": article_cible_id,
                                "Titre Article Cible": article_cible_name,
                                "Date de début (Article Cible)": article_cible_date_debut,
                                "Date de fin (Article Cible)": article_cible_date_fin
                            })

    return pd.DataFrame(rows)


In [102]:
df_2_2_3 = transform_json_to_dataframe(data_json_2_2_3)

In [103]:
def pandas_to_excel(df):
  df.to_excel('output.xlsx')
  return

In [104]:
pandas_to_excel(df_2_2_3)

# 3 - Concatenating all

Adding new columns by applying function 1 & 2

In [105]:
df_2_2_3.columns

Index(['Année', 'Version du', 'Date de début cible d'entrée en vigueur',
       'Est la dernière version', 'ID Article Modificateur',
       'Titre Article Modificateur', 'Nature Article Modificateur',
       'Date de début cible Article Modificateur',
       'Action Article Modificateur', 'ID Parent', 'Nom Parent', 'CID Parent',
       'ID Article Cible', 'Titre Article Cible',
       'Date de début (Article Cible)', 'Date de fin (Article Cible)'],
      dtype='object')

In [65]:
# STEP 1: add column 1 content of new version of articles 
# Utilisation du call API getArticle

def ajout_col_coutenu_NV(df):
    df['Contenu_Nouv_Vers_Article'] = df.apply( lambda x: getArticle(x['ID Article Cible']), axis=1)
    return display(df['Contenu_Nouv_Vers_Article'])

In [ ]:
# Temps d'attente important envir 10 min - 20 min
ajout_col_coutenu_NV(df_2_2_3)

In [76]:
# STEP 2: add column 2 content of old version of articles
# Utilisation du call API getArticle_prev_vers

def ajout_col_AV(df):
    df['Contenu_Ancien_Article'] = df.apply( lambda x: getArticle_prev_vers(x['ID Article Cible']), axis=1)
    return display(df['Contenu_Ancien_Article']) 

In [107]:
# Temps d'attente important envir 20 min 
ajout_col_AV(df_2_2_3)

0       I. – A droit à l'ouverture d'un compte de dépô...
1                                   KO version précédente
2       I. – Pour l'application de l'article L. 561-24...
3       I.-Sont applicables en Nouvelle-Calédonie, sou...
4       I.-Sont applicables en Polynésie française, so...
                              ...                        
2030    Le droit fixe dû au titre du I de l'article L....
2031    I. – Les fonds d'investissement de proximité s...
2032    I. – Sont applicables en Nouvelle-Calédonie le...
2033    I. – Sont applicables en Polynésie française l...
2034    I.-Sont applicables dans les îles Wallis et Fu...
Name: Contenu_Ancien_Article, Length: 2035, dtype: object

In [108]:
# STEP 3: add column 3 comparing old vs new version :

def compare_AV_vs_NV(df):
    df['Compare contenu'] = df.apply(
    lambda row: "Contenu Similaire" if row['Contenu_Nouv_Vers_Article'] == row['Contenu_Ancien_Article'] else "OK change",
    axis=1)
    return print(df['Compare contenu'])

In [109]:
compare_AV_vs_NV(df_2_2_3)

0               OK change
1               OK change
2               OK change
3       Contenu Similaire
4       Contenu Similaire
              ...        
2030            OK change
2031            OK change
2032            OK change
2033            OK change
2034            OK change
Name: Compare contenu, Length: 2035, dtype: object


In [80]:
# Final result : exporting to excel 
pandas_to_excel(df_2_2_3)